# Interpolating measurements from DEM images

In [1]:
import matplotlib.pyplot as plt
import preprocess
import operations
import linear_operations
import cv2
import numpy as np
import os
import re
import constants
from constants import ScarpClassification
from PIL import Image
from extract_data import interpolate_image
%load_ext autoreload
%autoreload 2

TensorFlow version: 2.13.0


### Create CSV


In [3]:
import datetime
def create_csv_header(): 
    day_dir = os.path.join("results", datetime.datetime.now().strftime("%m%d%Y"))
    if not os.path.exists(day_dir):
        os.makedirs(day_dir)
    cur_date = datetime.datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    file_to_write_to = os.path.join(day_dir, cur_date + "__measurements.csv")

    operations.write_row_to_csv(file_to_write_to, constants.HEADER, 'w')
    return file_to_write_to

### Most important part: Extracting the data from the image!

### Single Trial

In [51]:
pr_example = "D10_coh1.0e+06_ten1.0e+06_dip20_FS0.75"
mc_example = "D10_coh1.0e+06_ten1.0e+05_dip45_FS0.75"
ss_example = "D10_coh1.0e+06_ten1.0e+05_dip70_FS0.75"
trial = "M10_coh1.5e+06_ten1.0e+06_dip45_FS0.25"
bad_example = "L3_coh1.0e+05_ten1.0e+06_dip45_FS0.25"
hetero_ex = "D3_A_dip20_FS0.5"
bad_trial = "D10_R6_dip70_FS0.25"
trial = ss_example



# trial = "M5_coh5.0e+05_ten5.0e+05_dip70_FS0.5"
list_of_trials = [trial]
# list_of_trials = ["D5_coh1.5e+06_ten1.5e+06_dip60_FS0.25",
# "D5_coh1.5e+06_ten1.5e+06_dip60_FS0.5",
# "D5_coh1.5e+06_ten1.5e+06_dip60_FS0.75"]
number_of_images = 101
image_dict, trial_size_dict = preprocess.get_data_from_trials(number_of_images, use_all_trials = False, list_of_trials = list_of_trials, show_first_image = False)
print("This many images: ", len(image_dict))
# Commenting out so we dont write
#file_to_write_to = create_csv_header()

num_done = 0
total_images = len(image_dict.keys())
for image_path in list(image_dict.keys())[-4:]:
    test_image = image_dict[image_path]
    pixel_dictionary = interpolate_image(image_path, test_image, trial_size_dict, True, False, False)
    # Commenting out so we dont write
    #operations.put_dict_in_csv(file_to_write_to, pixel_dictionary)
    num_done+=1
    if num_done % 100 == 0:
        print("We did: ", num_done," out of ", total_images)
# print("wrote to: ", file_to_write_to)

trial_we_want = trial

img_dir = os.path.join("results", "visualizations", trial_we_want)

output_dir = os.path.join("results", "gifs")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# # Output GIF filename
# output_gif = trial_we_want + '.gif'
#TODO CHANGE THIS
output_gif = 'temp.gif'

# List all PNG files in the directory
png_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]
png_files.sort()  # Sort the files to maintain order

# Open the first image to get dimensions
with Image.open(os.path.join(img_dir, png_files[0])) as first_image:
    # Create a new GIF file and add frames
    gif_path = os.path.join(output_dir, output_gif)
    frames = [Image.open(os.path.join(img_dir, f)) for f in png_files[1:]]
    first_image.save(
        gif_path,
        save_all=True,
        append_images=frames,
        duration=200,  # Time in milliseconds for each frame
        loop=0  # 0 means the GIF will loop indefinitely
    )


This many images:  1818


### making viz for paper


In [2]:
paper_viz_trials = [
"D3_coh5.0e+05_ten5.0e+05_dip20_FS0.5",
"D5_coh5.0e+05_ten5.0e+05_dip20_FS0.5",
"D10_coh5.0e+05_ten5.0e+05_dip20_FS0.5",
"D3_coh1.0e+05_ten1.0e+05_dip60_FS0.25",
"D5_coh1.0e+05_ten1.0e+05_dip60_FS0.25",
"D10_coh1.0e+05_ten1.0e+05_dip60_FS0.25",
"D3_coh1.0e+06_ten1.0e+06_dip70_FS0.25",
"D5_coh1.5e+06_ten1.5e+06_dip60_FS0.75",
"D10_coh2.0e+06_ten1.0e+06_dip70_FS0.75",
"D3_coh2.0e+06_ten1.0e+06_dip20_FS0.25",
"D5_coh1.0e+06_ten1.0e+06_dip40_FS0.25",
"D10_coh1.0e+06_ten1.0e+06_dip20_FS0.25",
"D3_coh2.0e+06_ten2.0e+06_dip40_FS0.5",
"D5_coh2.0e+06_ten2.0e+06_dip40_FS0.5",
"D10_coh1.0e+06_ten1.0e+06_dip50_FS0.75",
"D3_coh1.5e+06_ten1.5e+06_dip40_FS0.5",
"D5_coh1.0e+06_ten1.0e+05_dip60_FS0.75",
"D10_coh1.0e+06_ten1.0e+05_dip70_FS0.75"
]

list_of_trials = paper_viz_trials

#error trials dont have contact folder, images are just wherever
interval_of_slip = 0.05
number_of_images = int(5/interval_of_slip) + 1
for i in range(len(list_of_trials)):
    print("starting trial ", list_of_trials[i])
    print("Total Progress ", i, "/", len(list_of_trials))
    single_image_dict, single_trial_size_dict = preprocess.get_data_from_trials(number_of_images, use_all_trials = False, list_of_trials = [list_of_trials[i]], show_first_image = False)
    for image_path in single_image_dict.keys():
        test_image = single_image_dict[image_path]
        if test_image is not None:
            pixel_dictionary = interpolate_image(image_path, test_image, single_trial_size_dict, True, False, False)
        else:
            print(f"Failed to load image: {image_path}")

starting trial  D3_coh5.0e+05_ten5.0e+05_dip20_FS0.5
Total Progress  0 / 18


c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))


starting trial  D5_coh5.0e+05_ten5.0e+05_dip20_FS0.5
Total Progress  1 / 18


c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))


starting trial  D10_coh5.0e+05_ten5.0e+05_dip20_FS0.5
Total Progress  2 / 18


...

c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\WillEnv\Lib\site-packages\sklearn\metrics\_regression.py:996: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


starting trial  D3_coh1.5e+06_ten1.5e+06_dip40_FS0.5
Total Progress  15 / 18


c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))


starting trial  D5_coh1.0e+06_ten1.0e+05_dip60_FS0.75
Total Progress  16 / 18
starting trial  D10_coh1.0e+06_ten1.0e+05_dip70_FS0.75
Total Progress  17 / 18


c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))
c:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\operations.py:546: RuntimeWarning: invalid value encountered in divide
  line_direction_normalized = np.array(line_direction) / np.linalg.norm(np.array(line_direction))


### All Trials (15 mins for 4500)

In [15]:
# file_to_write_to = r'C:\Users\wib994\WillCode\HarvardSeismo\ImageMeasuring\results\most_recent.csv'
file_to_write_to = create_csv_header()
# #TEMP, because we are adding missed trials

In [4]:
list_of_trials = preprocess.get_directories(r"I:\dem\Homogeneous_Results")
#TEMP, because we are adding missed trials:
file_to_write_to = r'C:\Users\wib994\WillCode\HarvardSeismo\most_recent-2.csv'
list_of_trials = ['M5_coh5.0e+05_ten1.0e+06_dip60_FS0.25', 'M5_coh5.0e+05_ten1.0e+06_dip60_FS0.5']
print("This how many trials: ", len(list_of_trials))

This how many trials:  2


In [52]:
#Find specific index of a trial
#problem w 893
# index = list_of_trials.index("M3_B_dip20_FS0.75")
# print(index)
file_to_write_to = r"C:\Users\wib994\WillCode\HarvardSeismo\good_trials_04_03.csv"
print(file_to_write_to)
list_of_trials = ['D10_R_dip20_FS0.25', 'D10_R_dip20_FS0.5', 'D10_R_dip20_FS0.75', 'D10_R_dip70_FS0.75', 'L10_K_dip20_FS0.25', 'L10_K_dip60_FS0.5', 'L3_A_dip20_FS0.25', 'L3_A_dip20_FS0.5', 'L3_A_dip20_FS0.75', 'L3_A_dip30_FS0.25', 'L3_A_dip30_FS0.5', 'L3_A_dip30_FS0.75', 'L3_A_dip40_FS0.25', 'L3_A_dip40_FS0.5', 'L3_A_dip40_FS0.75', 'M10_K_dip70_FS0.25', 'M10_K_dip70_FS0.5', 'M3_A_dip20_FS0.25', 'M3_A_dip20_FS0.5', 'M3_A_dip20_FS0.75', 'M5_G_dip40_FS0.25']

C:\Users\wib994\WillCode\HarvardSeismo\good_trials_04_03.csv


In [6]:
#error trials dont have contact folder, images are just wherever
interval_of_slip = 0.05
number_of_images = int(5/interval_of_slip) + 1
for i in range(len(list_of_trials)):
    print("starting trial ", list_of_trials[i])
    print("Total Progress ", i, "/", len(list_of_trials))
    single_image_dict, single_trial_size_dict = preprocess.get_data_from_trials(number_of_images, use_all_trials = False, list_of_trials = [list_of_trials[i]], show_first_image = False)
    for image_path in single_image_dict.keys():
        test_image = single_image_dict[image_path]
        if test_image is not None:
            pixel_dictionary = interpolate_image(image_path, test_image, single_trial_size_dict, False, False, False)
            # Commenting out so we don't write
            operations.put_dict_in_csv(file_to_write_to, pixel_dictionary)
        else:
            print(f"Failed to load image: {image_path}")

TensorFlow version: 2.13.0
starting trial  M5_coh5.0e+05_ten1.0e+06_dip60_FS0.25
Total Progress  0 / 2
starting trial  M5_coh5.0e+05_ten1.0e+06_dip60_FS0.5
Total Progress  1 / 2


In [ ]:
# Path to the file with unique 'Trial' values
bad_file_names_path = r"C:\Users\wib994\WillCode\HarvardSeismo\various_filtering_no_duplicates.txt"

# Read unique 'Trial' values from the bad file names file and store in list_of_trials
with open(bad_file_names_path, 'r') as bad_file:
    list_of_trials = [line.strip() for line in bad_file]

# Print the list of trials
print(list_of_trials)
print("This many bad trials: ", len(list_of_trials))

['M3_coh1.0e+06_ten1.0e+05_dip30_FS0.25']
This many bad trials:  1


In [ ]:
#FOR FIXING THE TRIALS W BAD NUMBERING
file_to_write_to = "good_trials_11_17.csv"
interval_of_slip = 0.05
number_of_images = int(5/interval_of_slip) + 1
for i, trial in enumerate(list_of_trials):
    print("starting trial ", trial)
    print("Total Progress ", i, "/", len(list_of_trials))
    single_image_dict, single_trial_size_dict = preprocess.get_data_from_trials(number_of_images, use_all_trials = False, list_of_trials = [trial], show_first_image = False)
    for image_path in single_image_dict.keys():
        test_image = single_image_dict[image_path]
        if test_image is not None:
            pixel_dictionary = interpolate_image(image_path, test_image, single_trial_size_dict, False)
            # Commenting out so we don't write
            operations.put_dict_in_csv(file_to_write_to, pixel_dictionary)
        else:
            print(f"Failed to load image: {image_path}")

starting trial  M3_coh1.0e+06_ten1.0e+05_dip30_FS0.25
Total Progress  0 / 1
